In [1]:
#!curl -fsSL https://ollama.com/install.sh | sh

In [2]:
# RUN IN TWO SEPARATE TERMINALS
#!ollama serve
#!ollama run llama3

In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")
llm.invoke("The first man on the moon was ...")

'The first men to walk on the moon were Neil Armstrong and Edwin "Buzz" Aldrin. They landed on July 20, 1969, as part of the Apollo 11 mission. Neil Armstrong was the first person to set foot on the moon\'s surface, famously declaring "That\'s one small step for man, one giant leap for mankind" as he stepped off the lunar module Eagle\'s ladder and onto the moon\'s dusty surface.'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings(model_name=model_name, gpt4all_kwargs=gpt4all_kwargs))

In [11]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [13]:
docs[0].page_content

'Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | llm | StrOutputParser()

# Run
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
chain.invoke(docs)

'The main themes in these documents are:\n\n1. **Task Decomposition**: The ability of Large Language Models (LLMs) to break down complex tasks into smaller, manageable subtasks using various prompting techniques.\n2. **Challenges in Long-term Planning and Task Decomposition**: The difficulties faced by LLMs when planning over a long period or adjusting plans in response to unexpected errors, making them less robust compared to humans.\n\nThese themes highlight the potential and limitations of using LLMs for complex tasks that require planning and problem-solving.'